In [37]:
import torch
import numpy as np
from mlp_mixer import MLPMixer

img = torch.ones([1, 3, 224, 224])

model = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=1000,
                 dim=512, depth=8, token_dim=256, channel_dim=2048)

parameters = filter(lambda p: p.requires_grad, model.parameters())
parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
print('Trainable Parameters: %.3fM' % parameters)

out_img = model(img)

print("Shape of out :", out_img.shape)  # [B, in_channels, image_size, image_size]

Trainable Parameters: 18.528M
Shape of out : torch.Size([1, 1000])


DataLoader

In [10]:
#pip install scikit-learn
!pip install numpy
#pip install 

DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [40]:
import os
import pickle
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from PIL import Image

class CIFAR10Dataset(Dataset):
    def __init__(self, data_dir, transform=None, train=True):
        self.data = []
        self.labels = []
        self.transform = transform

        # 훈련 데이터 파일 또는 테스트 데이터 파일 선택
        if train:
            for i in range(1, 3):
                file_path = os.path.join(data_dir, f'data_batch_{i}')
                with open(file_path, 'rb') as f:
                    batch = pickle.load(f, encoding='bytes')
                self.data.append(batch[b'data'])
                self.labels += batch[b'labels']
        else:
            file_path = os.path.join(data_dir, 'test_batch')
            with open(file_path, 'rb') as f:
                batch = pickle.load(f, encoding='bytes')
            self.data.append(batch[b'data'])
            self.labels += batch[b'labels']
        
        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img, label = self.data[idx], self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# 데이터 전처리 및 데이터셋 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'
train_dataset = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=True)
test_dataset = CIFAR10Dataset(data_dir=data_dir, transform=transform, train=False)

# 훈련 데이터셋을 훈련 및 검증 데이터셋으로 분할
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


In [41]:
print(len(train_dataset))  # 훈련 데이터셋의 크기
print(len(val_dataset))    # 검증 데이터셋의 크기
print(len(test_dataset))   # 테스트 데이터셋의 크기

first_data_point, first_label = train_dataset[0]
print(first_data_point.shape)


16000
4000
10000
torch.Size([3, 224, 224])


In [42]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [43]:
'''data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'  # 데이터셋 경로 수정
train_features, train_labels, test_features, test_labels = load_data(data_dir)

# 훈련 데이터를 훈련과 검증 데이터로 분할
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features, train_labels, test_size=0.1, random_state=42)'''


"data_dir = 'C:/Users/hongi/Desktop/MLP-Mixer-pytorch-master/cifar-10-batches-py'  # 데이터셋 경로 수정\ntrain_features, train_labels, test_features, test_labels = load_data(data_dir)\n\n# 훈련 데이터를 훈련과 검증 데이터로 분할\ntrain_features, val_features, train_labels, val_labels = train_test_split(\n    train_features, train_labels, test_size=0.1, random_state=42)"

In [44]:
'''batch_size = 64

# 데이터를 Tensor로 변환
train_features = torch.tensor(train_features)
train_labels = torch.tensor(train_labels)
val_features = torch.tensor(val_features)
val_labels = torch.tensor(val_labels)
test_features = torch.tensor(test_features)
test_labels = torch.tensor(test_labels)

# TensorDataset 생성
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)'''


'batch_size = 64\n\n# 데이터를 Tensor로 변환\ntrain_features = torch.tensor(train_features)\ntrain_labels = torch.tensor(train_labels)\nval_features = torch.tensor(val_features)\nval_labels = torch.tensor(val_labels)\ntest_features = torch.tensor(test_features)\ntest_labels = torch.tensor(test_labels)\n\n# TensorDataset 생성\ntrain_dataset = TensorDataset(train_features, train_labels)\nval_dataset = TensorDataset(val_features, val_labels)\ntest_dataset = TensorDataset(test_features, test_labels)\n\n# DataLoader 생성\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)\nval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)\ntest_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)'

model, loss func, optimizer

In [46]:
import torch.nn as nn
model = MLPMixer(in_channels=3, image_size=224, patch_size=16, num_classes=10,  # CIFAR-10 클래스 수
                 dim=512, depth=8, token_dim=256, channel_dim=2048)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train

In [47]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [50]:
from tqdm import tqdm

num_epochs = 150

best_val_accuracy = 0  # 가장 좋은 검증 정확도를 추적하기 위한 변수 초기화

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]')

    for inputs, labels in train_bar:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Validation]'):
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f'Validation Accuracy after epoch {epoch+1}: {val_accuracy:.2f}%')

    # 가장 좋은 검증 정확도를 기준으로 모델의 가중치 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model_weights.pth')  # 모델 가중치 저장
        print(f'New best model saved with accuracy: {best_val_accuracy:.2f}%')

print('Finished Training')

Epoch 1/150 [Training]: 100%|██████████| 250/250 [12:18<00:00,  2.95s/it]


Epoch 1, Loss: 1.2227


Epoch 1/150 [Validation]: 100%|██████████| 63/63 [01:10<00:00,  1.11s/it]


Validation Accuracy after epoch 1: 51.65%
New best model saved with accuracy: 51.65%


Epoch 2/150 [Training]: 100%|██████████| 250/250 [12:16<00:00,  2.94s/it]


...

Epoch 23/150 [Training]: 100%|██████████| 250/250 [12:20<00:00,  2.96s/it]


Epoch 23, Loss: 0.0641


Epoch 23/150 [Validation]: 100%|██████████| 63/63 [01:10<00:00,  1.11s/it]


Validation Accuracy after epoch 23: 52.85%


Epoch 24/150 [Training]:  64%|██████▍   | 161/250 [08:09<04:41,  3.16s/it]

In [ ]:
model.load_state_dict(torch.load('best_model_weights.pth'))
